In [1]:

# 📌 Milestone 1: Data Collection & Preprocessing
# Project: AI-Powered Enhanced EHR Imaging & Documentation System
# =====================================================

# -------------------------------
# Step 0: Install Required Libraries
# -------------------------------
!pip install -q kaggle pandas numpy pillow opencv-python pdfplumber pydicom



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.5 MB/s eta 0:00:00


In [2]:

# -------------------------------
# Step 1: Upload Kaggle API Key
# -------------------------------
from google.colab import files
uploaded = files.upload()  # Upload your kaggle.json here

# Move it to correct folder
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:

# -------------------------------
# Step 2: Create Project Folders
# -------------------------------
import os
folders = [
    "data/raw/images",
    "data/raw/notes",
    "data/images_processed",
    "data/ehr_notes_processed",
    "data"
]
for f in folders:
    os.makedirs(f, exist_ok=True)


In [4]:

# -------------------------------
# Step 3: Download Datasets from Kaggl
# -------------------------------

# 3a) COVID-19 Radiography Database (X-ray)
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database -p data/raw/images/
!unzip -q data/raw/images/covid19-radiography-database.zip -d data/raw/images/
!rm data/raw/images/covid19-radiography-database.zip

# 3b) Medical Transcriptions (EHR notes)
!kaggle datasets download -d tboyle10/medicaltranscriptions -p data/raw/notes/
!unzip -q data/raw/notes/medicaltranscriptions.zip -d data/raw/notes/
!rm data/raw/notes/medicaltranscriptions.zip

# Verify downloads
print("Sample images:", os.listdir("data/raw/images/")[:5])
print("Sample EHR notes:", os.listdir("data/raw/notes/")[:5])


Dataset URL: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database
License(s): copyright-authors
 98% 762M/778M [00:07<00:00, 199MB/s]
100% 778M/778M [00:07<00:00, 109MB/s]
Dataset URL: https://www.kaggle.com/datasets/tboyle10/medicaltranscriptions
License(s): CC0-1.0
  0% 0.00/4.85M [00:00<?, ?B/s]
100% 4.85M/4.85M [00:00<00:00, 910MB/s]
Sample images: ['COVID-19_Radiography_Dataset']
Sample EHR notes: ['mtsamples.csv']


In [5]:
# -------------------------------
# Step 4: Process Medical Images
# -------------------------------
from PIL import Image
import pydicom
import glob
import os

SRC_IMG = "data/raw/images/"
DST_IMG = "data/images_processed/"
TARGET = (256, 256)

os.makedirs(DST_IMG, exist_ok=True)

# Get all image files recursively
images = glob.glob(SRC_IMG + "**/*.png", recursive=True)
images += glob.glob(SRC_IMG + "**/*.jpg", recursive=True)
images += glob.glob(SRC_IMG + "**/*.jpeg", recursive=True)
images += glob.glob(SRC_IMG + "**/*.dcm", recursive=True)

print("Total images found:", len(images))

# Process and save
for path in images:
    try:
        fn = os.path.basename(path)
        if fn.lower().endswith('.dcm'):
            import pydicom
            ds = pydicom.dcmread(path)
            arr = ds.pixel_array.astype(float)
            arr = 255*(arr - arr.min())/(arr.max() - arr.min())
            img = Image.fromarray(arr.astype('uint8')).convert('L')
        else:
            img = Image.open(path).convert('L')
        img = img.resize(TARGET)
        outname = os.path.splitext(fn)[0] + '.png'
        img.save(os.path.join(DST_IMG, outname))
    except Exception as e:
        print("Skipping", path, "due to", e)

print("Processed images saved to:", DST_IMG)
print("Total processed images:", len(os.listdir(DST_IMG)))

Total images found: 42330
Processed images saved to: data/images_processed/
Total processed images: 21165
